In [ ]:
import requests
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import  KFold
import warnings
import re
warnings.filterwarnings('ignore')

In [ ]:
movies_db = pd.read_csv("movies_db_1.csv", delimiter=",",engine='python',  error_bad_lines=False) #Provient de 1_Recuperation_Movies

In [ ]:
movies_db.dropna(inplace=True) #Nettoyage des Na

In [ ]:
movies_db['actors'] = movies_db['actors'].str.strip('[]')
movies_db['director'] = movies_db['director'].str.strip('[]')
movies_db['producers'] = movies_db['producers'].str.strip('[]')

In [ ]:
def extract_popularities(col):
  movies_db_col = movies_db[col].apply(lambda s : re.findall('\'popularity\': \d+.?\d*',s))
  return movies_db_col.apply(lambda s :  np.sum([float(re.search('(\'popularity\': )(\d+.?\d*)', x).group(2)) for x in s]))

In [ ]:
movies_actors_popularities = movies_db['actors'].apply(lambda s : re.findall('\'popularity\': \d+.?\d*',s))

In [ ]:
movies_actors_popularities = movies_actors_popularities.apply(lambda s :  np.sum([float(re.search('(\'popularity\': )(\d+.?\d*)', x).group(2)) for x in s]))

In [ ]:
movies_db['pop_actors'] = extract_popularities('actors')
movies_db['pop_directors'] = extract_popularities('director')
movies_db['pop_producer'] = extract_popularities('producers')

In [ ]:
convert_dict = {'budget':'int64',
                'revenue':'int64',
                'runtime':'float64',
                'vote_count':'int64'
                }

movies_db[['budget','revenue','runtime','vote_count']] = movies_db[['budget','revenue','runtime','vote_count']].astype(convert_dict)

In [ ]:
movies_db['release_date'] = pd.to_datetime(movies_db['release_date'])

In [ ]:
movies_db.drop(movies_db[movies_db['revenue']<=0].index,inplace=True)

In [ ]:
movies_db.drop(movies_db[movies_db['runtime']==0].index,inplace=True)

In [ ]:
movies_db["year"] = movies_db["release_date"].apply(lambda s: s.year)

In [ ]:
movies_db.drop(movies_db[movies_db["year"]==2023].index,inplace=True)

In [ ]:
movies_db['director'] = movies_db['director'].str.strip('[]')
movies_db['producers'] = movies_db['producers'].str.strip('[]')
movies_db['genres'] = movies_db['genres'].str.strip('[]')
movies_db['production_companies'] = movies_db['production_companies'].str.strip('[]')

In [ ]:
movies_db['actors'] = movies_db['actors'].apply(lambda s : re.findall('[a-zA-Z]+ [a-zA-Z]*',s))

In [ ]:
movies_db = movies_db[movies_db['year']>=2010]

In [ ]:
movies_db.reset_index(inplace=True)
movies_db

,index,id,title,budget,genres,original_language,overview,production_companies,release_date,revenue,runtime,vote_average,vote_count,actors,director,producers,pop_actors,pop_directors,pop_producer,year
0,0,5,Four Rooms,4000000,"'Crime', 'Comedy'",en,It's Ted the Bellhop's first night on the job....,"'Miramax', 'A Band Apart'",1995-12-09,4257354,98.0,5.7,2086,"[Tim Roth, Tim Roth, Ted the, Jennifer Beals, ...","{'adult': False, 'gender': 1, 'id': 3110, 'kno...","{'adult': False, 'gender': 2, 'id': 2545, 'kno...",98.765,30.168,1.551,1995
1,1,6,Judgment Night,21000000,"'Action', 'Thriller', 'Crime'",en,"While racing to a boxing match, Frank, Mike, J...","'Universal Pictures', 'Largo Entertainment', '...",1993-10-15,12136938,110.0,6.5,224,"[Emilio Estevez, Emilio Estevez, Frank Wyatt, ...","{'adult': False, 'gender': 2, 'id': 2042, 'kno...","{'adult': False, 'gender': 2, 'id': 71417, 'kn...",61.835,2.359,0.600,1993
2,2,11,Star Wars,11000000,"'Adventure', 'Action', 'Science Fiction'",en,Princess Leia is captured and held hostage by ...,"'Lucasfilm', '20th Century Fox'",1977-05-25,775398007,121.0,8.2,16675,"[Mark Hamill, Mark Hamill, Luke Skywalker, Har...","{'adult': False, 'gender': 2, 'id': 1, 'known_...","{'adult': False, 'gender': 2, 'id': 12401, 'kn...",84.796,21.636,2.390,1977
3,3,12,Finding Nemo,94000000,"'Animation', 'Family'",en,"Nemo, an adventurous young clownfish, is unexp...",'Pixar',2003-05-30,940335536,100.0,7.8,15985,"[Albert Brooks, Albert Brooks, Marlin , Ellen ...","{'adult': False, 'gender': 2, 'id': 7, 'known_...","{'adult': False, 'gender': 2, 'id': 9, 'known_...",99.258,9.682,1.400,2003
4,4,13,Forrest Gump,55000000,"'Comedy', 'Drama', 'Romance'",en,A man with a low IQ has accomplished great thi...,"'Paramount', 'The Steve Tisch Company'",1994-07-06,677387716,142.0,8.5,22143,"[Tom Hanks, Tom Hanks, Forrest Gump, Robin Wri...","{'adult': False, 'gender': 2, 'id': 24, 'known...","{'adult': False, 'gender': 1, 'id': 28, 'known...",107.316,3.925,5.474,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7088,7124,392982,Marshall,12000000,'Drama',en,"Thurgood Marshall, the first African-American ...",'Chestnut Ridge',2017-10-13,10051659,118.0,7.3,479,"[Chadwick Boseman, Chadwick Boseman, Thurgood ...","{'adult': False, 'gender': 2, 'id': 68215, 'kn...","{'adult': False, 'gender': 1, 'id': 9183, 'kno...",64.272,1.868,5.625,2017
7089,7125,261949,Thanks Maa,500000,"'Comedy', 'Drama'",en,"While on the run from the reformatory, a 12 ye...","'Quantum Films [in]', 'Sony Pictures'",2010-03-05,2000000,120.0,5.8,3,"[Master Shams, Master Shams, Munsipalty Ghatko...","{'adult': False, 'gender': 0, 'id': 1265938, '...",,12.806,1.214,0.000,2010
7090,7126,78127,Silk Shoes,300,"'Comedy', 'Drama'",en,With his film having failed and producer havin...,,2006-06-22,300,100.0,4.9,4,"[Choi Deok, Choi Deok, hye Kim, hye Kim, Lee S...","{'adult': False, 'gender': 2, 'id': 96780, 'kn...","{'adult': False, 'gender': 2, 'id': 96780, 'kn...",19.557,0.732,0.732,2006
7091,7127,655232,Vazante,2424713,'Drama',en,"On an empty beach, 5 kids talk and talk, over ...",'Processo Multiartes',2018-06-10,100,18.0,0.0,0,[],"{'adult': False, 'gender': 2, 'id': 1270398, '...","{'adult': False, 'gender': 0, 'id': 1270396, '...",0.000,0.840,0.600,2018


In [ ]:
movies_db.drop(["index"],axis=1,inplace=True)

Importation données Youtube

In [ ]:
youtube = pd.read_csv("Film_View_Like_BIS.csv",engine='python',names=["count_view","count_like"],sep=None)

In [ ]:
youtube.reset_index(inplace=True)

In [ ]:
youtube.rename(columns={"index":"title"},inplace=True)

In [ ]:
movies_db['title']=movies_db['title'].apply(lambda s: s.replace(":"," "))
movies_db['title']=movies_db['title'].apply(lambda s: s.replace("-"," "))
youtube['title']=youtube['title'].apply(lambda s: s.replace('"',''))
youtube['title']=youtube['title'].apply(lambda s: s.replace(":"," "))
youtube['title']=youtube['title'].apply(lambda s: s.replace("-"," "))

In [ ]:
movies_db=pd.merge(movies_db,youtube,on=["title"])

Export CSV

In [26]:
movies_db[["id","title","budget","revenue","runtime","pop_actors","vote_average","vote_count","pop_directors","pop_producer"]].to_csv("movies_db_2.csv",index=False)